# Shamir Secret Sharing

**Module 12** | 12-mpc

*Polynomial interpolation, t-of-n*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **shamir secret sharing**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Basic familiarity with Python syntax.
- A working SageMath installation (or access to CoCalc/SageMathCell).

## The Idea: Hide a Secret in a Polynomial

In [ ]:
# Secret s is the constant term of a random degree-(t-1) polynomial
# Share i = f(i) for i = 1, 2, ..., n
R.<x> = PolynomialRing(GF(next_prime(1000)))
# TODO: construct a random polynomial with secret as f(0)

## Sharing

In [ ]:
# Evaluate the polynomial at n distinct points
# Any t shares can reconstruct; fewer than t reveal nothing
# TODO: generate n shares from the polynomial

## Reconstruction via Lagrange Interpolation

In [ ]:
# Given t shares (x_i, y_i), interpolate to recover f(0)
# TODO: use Lagrange interpolation to reconstruct the secret

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **shamir secret sharing**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Additive Secret Sharing](12b-secret-sharing-additive.ipynb)